In [1]:
import pandas as pd

In [2]:
train_bodies = pd.read_csv('./train_bodies.csv')
train_stances = pd.read_csv('./train_stances.csv')

In [3]:
train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [4]:
train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [5]:
train_bodies[train_bodies['Body ID'] == 712]

,Body ID,articleBody
475,712,Danny Boyle is directing the untitled film\n\n...


In [6]:
merged = pd.merge(train_stances, train_bodies, on='Body ID')
merged.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss,Danny Boyle is directing the untitled film\n\n...
2,Mexico police find mass grave near site 43 stu...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
3,Mexico Says Missing Students Not Found In Firs...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
4,New iOS 8 bug can delete all of your iCloud do...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [7]:
merged.shape

(49972, 4)

In [8]:
merged.drop_duplicates().shape

(49570, 4)

In [9]:
train = merged

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [11]:
bow_vect = CountVectorizer(max_features=5000).fit(train['Headline'] + train['articleBody'])

In [12]:
train_body = bow_vect.transform(train['articleBody']).toarray()

In [13]:
train_headline = bow_vect.transform(train['Headline']).toarray()

In [14]:
import numpy as np

In [15]:
train_features = np.concatenate([train_headline, train_body], axis=1)

In [16]:
train_features.shape

(49972, 10000)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
model = LogisticRegression().fit(train_features, train['Stance'])

In [19]:
test = pd.read_csv('data/test/test.csv')

In [20]:
test.shape

(25413, 4)

In [21]:
test = test.drop_duplicates()

In [22]:
test.shape

(25171, 4)

In [23]:
test_body = bow_vect.transform(test['articleBody']).toarray()
test_headline = bow_vect.transform(test['Headline']).toarray()

In [24]:
test_body.shape, test_headline.shape

((25171, 5000), (25171, 5000))

In [25]:
test_features = np.concatenate([test_headline, test_body], axis=1)

In [26]:
pred = model.predict(test_features)

In [27]:
from sklearn.metrics import classification_report,confusion_matrix

In [28]:
print(classification_report(y_pred=pred, y_true=test['Stance']))
print(confusion_matrix(y_pred=pred, y_true=test['Stance']))

             precision    recall  f1-score   support

      agree       0.13      0.20      0.16      1886
   disagree       0.00      0.00      0.00       697
    discuss       0.21      0.40      0.28      4406
  unrelated       0.72      0.56      0.63     18182

avg / total       0.57      0.49      0.52     25171

[[  372     2   374  1138]
 [  121     0   126   450]
 [  353     0  1759  2294]
 [ 2053     0  5923 10206]]
